# Retrieval-Augmented Generation with Vector Stores


### **Environment Setup:**

In [7]:
%pip install -q langchain langchain-community langchain-nvidia-ai-endpoints gradio rich
%pip install -q arxiv pymupdf faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00


In [11]:
import os
os.environ["NVIDIA_API_KEY"] = "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"

In [12]:
from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
pprint = partial(console.print, style=base_style)

In [13]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x22b-instruct-v0.1")

## 1: Summary of RAG Workflows

This notebook will explore several paradigms and derive reference code to help you approach some of the most common retrieval-augmented workflows. Specifically, the following sections will be covered (with the differences highlighted):

<br>

> ***Vector Store Workflow for Conversational Exchanges:***
- Generate semantic embedding for each new conversation.
- Add the message body to a vector store for retrieval.
- Query the vector store for relevant messages to fill in the LLM context.

<br>

> ***Modified Workflow for an Arbitrary Document:***
- **Divide the document into chunks and process them into useful messages.**
- Generate semantic embedding for each **new document chunk**.
- Add the **chunk bodies** to a vector store for retrieval.
- Query the vector store for relevant **chunks** to fill in the LLM context.
    - ***Optional:* Modify/synthesize results for better LLM results.**

<br>

> **Extended Workflow for a Directory of Arbitrary Documents:**
- Divide **each document** into chunks and process them into useful messages.
- Generate semantic embedding for each new document chunk.
- Add the chunk bodies to **a scalable vector database for fast retrieval**.
    - ***Optional*: Exploit hierarchical or metadata structures for larger systems.**
- Query the **vector database** for relevant chunks to fill in the LLM context.
    - *Optional:* Modify/synthesize results for better LLM results.

<br>

Some of the most important terminology surrounding RAG is covered in detail on the [**LlamaIndex Concepts page**](https://docs.llamaindex.ai/en/stable/getting_started/concepts.html), which itself is a great starting point for progressing towards the LlamaIndex loading and retrieving strategy. We highly recommend using it as a reference as you continue with this notebook and advise you to try out LlamaIndex after the course to consider the pros and cons firsthand!


<!-- > <img src="https://drive.google.com/uc?export=view&id=1cFbKbVvLLnFPs3yWCKIuzXkhBWh6nLQY" width=1200px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/data_connection_langchain.jpeg" width=1200px/>
>
> From [**Retrieval | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/)

## **2:** RAG for Conversation History

In our previous explorations, we delved into the capabilities of document embedding models and used them to embed, store, and compare semantic vector representations of text. Though we could motivate how to efficiently extend this into vector store land manually, the true beauty of working with a standard API is its strong incorporation with other frameworks that can already do the heavy lifting for us!

<br>

### **Step 1**: Getting A Conversation

Consider a conversation crafted using Llama-13B between a chat agent and a blue bear named Beras. This dialogue, dense with details and potential diversions, provides a rich dataset for our study:


In [14]:
conversation = [  ## This conversation was generated partially by an AI system, and modified to exhibit desirable properties
    "[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?",
    "[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch across North America",
    "[Beras] Wow, that sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.",
    "[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you!"
    "[Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.",
    "[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching documentaries about them."
    "[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just curious, ya know!",
    "[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains and their significance!"
]

Using the manual embedding strategy from the previous notebook is still very viable, but we can also rest easy and let a **vector store** do all that work for us!

<br>

### **Step 2:** Constructing Our Vector Store Retriever

To streamline similarity queries on our conversation, we can employ a vector store to help keep track of passages for us! **Vector Stores**, or vector storage systems, abstract away most of the low-level details of the embedding/comparison strategies and provide a simple interface to load and compare vectors.


<!-- > <img src="https://drive.google.com/uc?export=view&id=1ZjwYbSZzsXK6ZP8O1-cY3BeRffV4oqzb" width=1000px/> -->
> <img src="https://dli-lms.s3.amazonaws.com/assets/s-fx-15-v1/imgs/vector_stores.jpeg" width=1200px/>
>
> From [**Vector Stores | LangChain**🦜️🔗](https://python.langchain.com/docs/modules/data_connection/vectorstores/)

<br>

In addition to simplifying the process from an API perspective, vector stores also implement connectors, integrations, and optimizations under the hood. In our case, we will start with the [**FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss), which integrates a LangChain-compatable Embedding model with the [**FAISS (Facebook AI Similarity Search)**](https://github.com/facebookresearch/faiss) library to make the process fast and scalable on our local machine!

**Specifically:**

1. We can feed our conversation into [**a FAISS vector store**](https://python.langchain.com/docs/integrations/vectorstores/faiss) via the `from_texts` constructor. This will take our conversational data and the embedding model to create a searchable index over our discussion.
2. This vector store can then be "interpreted" as a retriever, supporting the LangChain runnable API and returning documents retrieved via an input query.

The following shows how you can construct a FAISS vector store and reinterpret it as a retriever using the LangChain `vectorstore` API:

In [15]:
%%time
## ^^ This cell will be timed to see how long the conversation embedding takes
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain.vectorstores import FAISS

## Streamlined from_texts FAISS vector store construction from text list
convstore = FAISS.from_texts(conversation, embedding=embedder)
retriever = convstore.as_retriever()

CPU times: user 55.8 ms, sys: 9 ms, total: 64.8 ms
Wall time: 1.23 s


In [16]:
pprint(retriever.invoke("Who are you?"))

[
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    ),
    Document(
        page_content='[Agent] I hope you get to visit them someday, Beras! It would be a great adventure for 
you![Beras] Thank you for the suggestion! Ill definitely keep it in mind for the future.'
    ),
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    )
]

In [17]:
pprint(retriever.invoke("Where are the Rocky Mountains?"))

[
    Document(
        page_content='[Agent] The Rocky Mountains are a beautiful and majestic range of mountains that stretch 
across North America'
    ),
    Document(
        page_content="[Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research 
online or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate 
there. I was just curious, ya know!"
    ),
    Document(
        page_content="[User]  Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the 
rocky mountains?"
    ),
    Document(
        page_content='[Agent] Absolutely! Lets continue the conversation and explore more about the Rocky Mountains
and their significance!'
    )
]

As we can see, our retriever found a handful of semantically relevant documents from our query. You may notice that not all of the documents are useful or clear on their own. For example, a retrieval of *"Beras"* for *"your name"* may be problematic for the chatbot if provided out of context. Anticipating the potential problems and creating synergies between your LLM components can increase the likelihood of good RAG behavior, so keep an eye out for such pitfalls and opportunities.

<br>

### **Step 3:** Incorporating Conversation Retrieval Into Our Chain

Now that we have our loaded retriever component as a chain, we can incorporate it into our existing chat system as before. Specifically, we can start with an ***always-on RAG formulation*** where:
- **A retriever is always retrieving context by default**.
- **A generator is acting on the retrieved context**.

In [18]:
from langchain.document_transformers import LongContextReorder
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain.schema.runnable.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

########################################################################
## Utility Runnables/Methods
def RPrint(preface=""):
    """Simple passthrough "prints, then returns" chain"""
    def print_and_return(x, preface):
        if preface: print(preface, end="")
        pprint(x)
        return x
    return RunnableLambda(partial(print_and_return, preface=preface))

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name:
            out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

## Optional; Reorders longer documents to center of output text
long_reorder = RunnableLambda(LongContextReorder().transform_documents)

In [19]:
context_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {question}"
    "\nAnswer the user conversationally. User is not aware of context."
)

chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'question': (lambda x:x)
    }
    | context_prompt
    | RPrint()
    | instruct_llm
    | StrOutputParser()
)

pprint(chain.invoke("Where does Beras live?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [User] 
Hello! My name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?\n[Quote from 
Document] [Agent] I hope you get to visit them someday, Beras! It would be a great adventure for you![Beras] Thank 
you for the suggestion! Ill definitely keep it in mind for the future.\n[Quote from Document] [Beras] Wow, that 
sounds amazing! Ive never been to the Rocky Mountains before, but Ive heard many great things about them.\n[Quote 
from Document] [Agent] In the meantime, you can learn more about the Rocky Mountains by doing some research online 
or watching documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate there. I was
just curious, ya know!\n\n\nUser Question: Where does Beras live?\nAnswer the user conversationally. User is not 
aware of context."
        )
    ]
)

Based on the context provided, Beras lives in the Arctic. It seems like the Rocky Mountains are a fascinating place
for Beras to learn about, considering the vastly different climate from where they currently reside!

Take a second to try out some more invocations and see how the new setup performs. Regardless of your model choice, the following questions should serve as interesting starting points.

In [20]:
pprint(chain.invoke("Where are the Rocky Mountains?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [Agent]
In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching 
documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just 
curious, ya know!\n[Quote from Document] [Agent] Absolutely! Lets continue the conversation and explore more about 
the Rocky Mountains and their significance!\n[Quote from Document] [User]  Hello! My name is Beras, and I'm a big 
blue bear! Can you please tell me about the rocky mountains?\n[Quote from Document] [Agent] The Rocky Mountains are
a beautiful and majestic range of mountains that stretch across North America\n\n\nUser Question: Where are the 
Rocky Mountains?\nAnswer the user conversationally. User is not aware of context."
        )
    ]
)

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. Unfortunately, 
without more context, I can't give you a precise location beyond that. However, you can find out more by doing some
research online or watching documentaries about them. I hope you enjoy learning about the Rocky Mountains!

In [21]:
pprint(chain.invoke("Where are the Rocky Mountains? Are they close to California?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [Agent]
In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching 
documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just 
curious, ya know!\n[Quote from Document] [Agent] Absolutely! Lets continue the conversation and explore more about 
the Rocky Mountains and their significance!\n[Quote from Document] [User]  Hello! My name is Beras, and I'm a big 
blue bear! Can you please tell me about the rocky mountains?\n[Quote from Document] [Agent] The Rocky Mountains are
a beautiful and majestic range of mountains that stretch across North America\n\n\nUser Question: Where are the 
Rocky Mountains? Are they close to California?\nAnswer the user conversationally. User is not aware of context."
        )
    ]
)

Hello there! The Rocky Mountains are a stunning range of mountains that span across North America. To answer your 
question about their location, they do not directly border California. In fact, they run north to south, starting 
from western Canada and stretching all the way down through the states of Montana, Idaho, Wyoming, Colorado, and 
into New Mexico. So, while they're not close to California, they certainly are a significant part of the North 
American landscape. Is there anything else you'd like to know about them?

In [22]:
pprint(chain.invoke("How far away is Beras from the Rocky Mountains?"))

ChatPromptValue(
    messages=[
        HumanMessage(
            content="Answer the question using only the context\n\nRetrieved Context: [Quote from Document] [Agent]
In the meantime, you can learn more about the Rocky Mountains by doing some research online or watching 
documentaries about them.[Beras] I live in the arctic, so I'm not used to the warm climate there. I was just 
curious, ya know!\n[Quote from Document] [Agent] The Rocky Mountains are a beautiful and majestic range of 
mountains that stretch across North America\n[Quote from Document] [Beras] Wow, that sounds amazing! Ive never been
to the Rocky Mountains before, but Ive heard many great things about them.\n[Quote from Document] [User]  Hello! My
name is Beras, and I'm a big blue bear! Can you please tell me about the rocky mountains?\n\n\nUser Question: How 
far away is Beras from the Rocky Mountains?\nAnswer the user conversationally. User is not aware of context."
        )
    ]
)

Unfortunately, the context doesn't provide information on Beras' exact location or distance from the Rocky 
Mountains. I can tell you, though, that the Rocky Mountains are a beautiful and majestic range of mountains 
stretching across North America. If you'd like, I can search for more information about their geographical location
and specific features.

<br>

You might notice some decent performance with this always-on retrieval node in the loop since the actual context being fed into the LLM remains relatively small. It's important to experiment with factors like embedding sizes, context limits, and model options to see what kinds of behavior you can expect and which efforts are worth taking to improve performance.

<br>

### **Step 4:** Automatic Conversation Storage

Now that we see how our vector store memory unit should function, we can perform one last integration to allow our conversation to add new entries to our conversation: a runnable that calls the `add_texts` method for us to update the store state.


In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

## Reset knowledge base and define what it means to add more messages.
convstore = FAISS.from_texts(conversation, embedding=embedder)

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([f"User said {d.get('input')}", f"Agent said {d.get('output')}"])
    return d.get('output')

chat_prompt = ChatPromptTemplate.from_template(
    "Answer the question using only the context"
    "\n\nRetrieved Context: {context}"
    "\n\nUser Question: {input}"
    "\nAnswer the user conversationally. Make sure the conversation flows naturally.\n"
    "[Agent]"
)


conv_chain = (
    {
        'context': convstore.as_retriever() | long_reorder | docs2str,
        'input': (lambda x:x)
    }
    | RunnableAssign({'output' : chat_prompt | instruct_llm | StrOutputParser()})
    | partial(save_memory_and_get_output, vstore=convstore)
)

pprint(conv_chain.invoke("I'm glad you agree! I can't wait to get some ice cream there! It's such a good food!"))
print()
pprint(conv_chain.invoke("Can you guess what my favorite food is?"))
print()
pprint(conv_chain.invoke("Actually, my favorite is honey! Not sure where you got that idea?"))
print()
pprint(conv_chain.invoke("I see! Fair enough! Do you know my favorite food now?"))

While I'm sure the Rocky Mountains would provide a magnificent backdrop for enjoying ice cream, it's essential to 
remember that maintaining the pristine beauty of natural wonders like the Rockies is crucial. During our 
exploration of the Rocky Mountains, we'll discuss their impressive features, wildlife, and how best to appreciate 
them while minimizing our environmental impact. And who knows, perhaps on your future visit, you'll find a 
delightful spot to enjoy a sweet treat amidst the breathtaking scenery.

While we've been discussing the Rocky Mountains and the environmental considerations, I do recall a mention about a
beloved food of yours! You were quite enthusiastic about having some ice cream there. Given your excitement, it 
seems safe to guess that ice cream might just be your favorite food. Would I be correct?

[Agent] Oh, I see! I must have gotten mixed up earlier. It seems I incorrectly guessed that your favorite food was 
ice cream based on our conversation about enjoying a sweet treat in the Rockies. However, now that you've mentioned
it, honey does sound absolutely delightful. It's quite fascinating, isn't it, how a simple ingredient like honey 
can be used in so many ways and dishes? I'm sure it adds a special touch to many of your favorite meals. Would you 
like to discuss more about honey or anything else you're interested in?

Yes, I do! I apologize for the mix-up earlier. I now know that your favorite food is indeed honey. It must be quite
a versatile ingredient to be used in so many ways and dishes! If you'd like, we can continue to discuss honey or 
other topics that interest you.

Unlike the more automatic full-text or rule-based approaches to injecting context into the LLM, this approach ensures some amount of consolidation which can keep the context length from getting out of hand. It's still not a full-proof strategy on its own, but it's a stark improvement for unstructured conversations (and doesn't even require a strong instruction-tuned model to perform slot-filling).

----

<br>

## **3:** RAG For Document Chunk Retrieval

Given our prior exploration of document loading, the idea that data chunks can be embedded and searched through probably isn't surprising. With that said, it is definitely worth going over since applying RAG with documents is a double-edged sword; it may **seem** to work well out of the box but requires some extra care when optimizing it for truly reliable performance. It also provides an excellent opportunity to review some fundamental LCEL skills, so let's see what we can do!


<br>

### **Task 1**: Loading And Chunking Your Documents

A few simplifying assumptions and additional processing steps are included to help you improve your naive RAG performance:

- Documents are cut off prior to the "References" section if one exists. This will keep our system from considering the citations and appendix sections, which tend to be long and distracting.

- A chunk that lists the available documents is inserted to provide a high-level view of all available documents in a single chunk. If your pipeline does not provide metadata on each retrieval, this is a useful component and can even be listed among a list of higher-priority pieces if appropriate.

- Additionally, the metadata entries are also inserted to provide general information. Ideally, there would also be some synthetic chunks that merge the metadata into interesting cross-document chunks.


In [24]:
import json
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import ArxivLoader

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100,
    separators=["\n\n", "\n", ".", ";", ",", " "],
)


print("Loading Documents")
docs = [
    ArxivLoader(query="1706.03762").load(),  ## Attention Is All You Need Paper
    ArxivLoader(query="1810.04805").load(),  ## BERT Paper
    ArxivLoader(query="2005.11401").load(),  ## RAG Paper
    ArxivLoader(query="2205.00445").load(),  ## MRKL Paper
    ArxivLoader(query="2310.06825").load(),  ## Mistral Paper
    ArxivLoader(query="2306.05685").load(),  ## LLM-as-a-Judge
    ## Some longer papers
    ArxivLoader(query="2210.03629").load(),  ## ReAct Paper
    ArxivLoader(query="2112.10752").load(),  ## Latent Stable Diffusion Paper
    ArxivLoader(query="2103.00020").load(),  ## CLIP Paper
    ArxivLoader(query="2408.11788").load(), ## DreamFactory
]

## Cut the paper short if references is included.
## This is a standard string in papers.
for doc in docs:
    content = json.dumps(doc[0].page_content)
    if "References" in content:
        doc[0].page_content = content[:content.index("References")]

## Split the documents and also filter out stubs (overly short chunks)
print("Chunking Documents")
docs_chunks = [text_splitter.split_documents(doc) for doc in docs]
docs_chunks = [[c for c in dchunks if len(c.page_content) > 200] for dchunks in docs_chunks]

## Make some custom Chunks to give big-picture details
doc_string = "Available Documents:"
doc_metadata = []
for chunks in docs_chunks:
    metadata = getattr(chunks[0], 'metadata', {})
    doc_string += "\n - " + metadata.get('Title')
    doc_metadata += [str(metadata)]

extra_chunks = [doc_string] + doc_metadata

## Printing out some summary information for reference
pprint(doc_string, '\n')
for i, chunks in enumerate(docs_chunks):
    print(f"Document {i}")
    print(f" - # Chunks: {len(chunks)}")
    print(f" - Metadata: ")
    pprint(chunks[0].metadata)
    print()

Loading Documents
Chunking Documents


Available Documents:
 - Attention Is All You Need
 - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
 - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks
 - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge 
sources and discrete reasoning
 - Mistral 7B
 - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena
 - ReAct: Synergizing Reasoning and Acting in Language Models
 - High-Resolution Image Synthesis with Latent Diffusion Models
 - Learning Transferable Visual Models From Natural Language Supervision
 - DreamFactory: Pioneering Multi-Scene Long Video Generation with a Multi-Agent Framework 

Document 0
 - # Chunks: 35
 - Metadata: 


{
    'Published': '2023-08-02',
    'Title': 'Attention Is All You Need',
    'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz 
Kaiser, Illia Polosukhin',
    'Summary': 'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural 
networks in an encoder-decoder configuration. The best\nperforming models also connect the encoder and decoder 
through an attention\nmechanism. We propose a new simple network architecture, the Transformer, based\nsolely on 
attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation
tasks show these models to be\nsuperior in quality while being more parallelizable and requiring 
significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014\nEnglish-to-German translation 
task, improving over the existing best results,\nincluding ensembles by over 2 BLEU. On the WMT 2014 
English-to-French\ntranslation task, our model establishes a new single-model state-of-the-art\nBLEU score of 41.8 
after training for 3.5 days on eight GPUs, a small fraction\nof the training costs of the best models from the 
literature. We show that the\nTransformer generalizes well to other tasks by applying it successfully to\nEnglish 
constituency parsing both with large and limited training data.'
}


Document 1
 - # Chunks: 45
 - Metadata: 


...

{
    'Published': '2022-04-13',
    'Title': 'High-Resolution Image Synthesis with Latent Diffusion Models',
    'Authors': 'Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer',
    'Summary': 'By decomposing the image formation process into a sequential application of\ndenoising 
autoencoders, diffusion models (DMs) achieve state-of-the-art\nsynthesis results on image data and beyond. 
Additionally, their formulation\nallows for a guiding mechanism to control the image generation process 
without\nretraining. However, since these models typically operate directly in pixel\nspace, optimization of 
powerful DMs often consumes hundreds of GPU days and\ninference is expensive due to sequential evaluations. To 
enable DM training on\nlimited computational resources while retaining their quality and flexibility,\nwe apply 
them in the latent space of powerful pretrained autoencoders. In\ncontrast to previous work, training diffusion 
models on such a representation\nallows for the first time to reach a near-optimal point between 
complexity\nreduction and detail preservation, greatly boosting visual fidelity. By\nintroducing cross-attention 
layers into the model architecture, we turn\ndiffusion models into powerful and flexible generators for general 
conditioning\ninputs such as text or bounding boxes and high-resolution synthesis becomes\npossible in a 
convolutional manner. Our latent diffusion models (LDMs) achieve\na new state of the art for image inpainting and 
highly competitive performance\non various tasks, including unconditional image generation, semantic 
scene\nsynthesis, and super-resolution, while significantly reducing computational\nrequirements compared to 
pixel-based DMs. Code is available at\nhttps://github.com/CompVis/latent-diffusion .'
}


Document 8
 - # Chunks: 155
 - Metadata: 


{
    'Published': '2021-02-26',
    'Title': 'Learning Transferable Visual Models From Natural Language Supervision',
    'Authors': 'Alec Radford, Jong Wook Kim, Chris Hallacy, Aditya Ramesh, Gabriel Goh, Sandhini Agarwal, Girish 
Sastry, Amanda Askell, Pamela Mishkin, Jack Clark, Gretchen Krueger, Ilya Sutskever',
    'Summary': 'State-of-the-art computer vision systems are trained to predict a fixed set\nof predetermined 
object categories. This restricted form of supervision limits\ntheir generality and usability since additional 
labeled data is needed to\nspecify any other visual concept. Learning directly from raw text about images\nis a 
promising alternative which leverages a much broader source of\nsupervision. We demonstrate that the simple 
pre-training task of predicting\nwhich caption goes with which image is an efficient and scalable way to 
learn\nSOTA image representations from scratch on a dataset of 400 million (image,\ntext) pairs collected from the 
internet. After pre-training, natural language\nis used to reference learned visual concepts (or describe new ones)
enabling\nzero-shot transfer of the model to downstream tasks. We study the performance\nof this approach by 
benchmarking on over 30 different existing computer vision\ndatasets, spanning tasks such as OCR, action 
recognition in videos,\ngeo-localization, and many types of fine-grained object classification. The\nmodel 
transfers non-trivially to most tasks and is often competitive with a\nfully supervised baseline without the need 
for any dataset specific training.\nFor instance, we match the accuracy of the original ResNet-50 on 
ImageNet\nzero-shot without needing to use any of the 1.28 million training examples it\nwas trained on. We release
our code and pre-trained model weights at\nhttps://github.com/OpenAI/CLIP.'
}


Document 9
 - # Chunks: 41
 - Metadata: 


{
    'Published': '2024-08-21',
    'Title': 'DreamFactory: Pioneering Multi-Scene Long Video Generation with a Multi-Agent Framework',
    'Authors': 'Zhifei Xie, Daniel Tang, Dingwei Tan, Jacques Klein, Tegawend F. Bissyand, Saad Ezzini',
    'Summary': 'Current video generation models excel at creating short, realistic clips, but\nstruggle with 
longer, multi-scene videos. We introduce \\texttt{DreamFactory},\nan LLM-based framework that tackles this 
challenge. \\texttt{DreamFactory}\nleverages multi-agent collaboration principles and a Key Frames 
Iteration\nDesign Method to ensure consistency and style across long videos. It utilizes\nChain of Thought (COT) to
address uncertainties inherent in large language\nmodels. \\texttt{DreamFactory} generates long, stylistically 
coherent, and\ncomplex videos. Evaluating these long-form videos presents a challenge. We\npropose novel metrics 
such as Cross-Scene Face Distance Score and Cross-Scene\nStyle Consistency Score. To further research in this area,
we contribute the\nMulti-Scene Videos Dataset containing over 150 human-rated videos.'
}

<br>

### **Task 2**: Construct Your Document Vector Stores

Now that we have all of the components, we can go ahead and create indices surrounding them:

In [26]:
%%time
print("Constructing Vector Stores")
vecstores = [FAISS.from_texts(extra_chunks, embedder)]
vecstores += [FAISS.from_documents(doc_chunks, embedder) for doc_chunks in docs_chunks]

Constructing Vector Stores
CPU times: user 1.84 s, sys: 113 ms, total: 1.96 s
Wall time: 1min 10s


In [25]:
extra_chunks

['Available Documents:\n - Attention Is All You Need\n - BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding\n - Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks\n - MRKL Systems: A modular, neuro-symbolic architecture that combines large language models, external knowledge sources and discrete reasoning\n - Mistral 7B\n - Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena\n - ReAct: Synergizing Reasoning and Acting in Language Models\n - High-Resolution Image Synthesis with Latent Diffusion Models\n - Learning Transferable Visual Models From Natural Language Supervision\n - DreamFactory: Pioneering Multi-Scene Long Video Generation with a Multi-Agent Framework',
 "{'Published': '2023-08-02', 'Title': 'Attention Is All You Need', 'Authors': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin', 'Summary': 'The dominant sequence transduction models are based on complex r

In [27]:
vecstores

In [28]:
from faiss import IndexFlatL2
from langchain_community.docstore.in_memory import InMemoryDocstore

embed_dims = len(embedder.embed_query("test"))
def default_FAISS():
    '''Useful utility for making an empty FAISS vectorstore'''
    return FAISS(
        embedding_function=embedder,
        index=IndexFlatL2(embed_dims),
        docstore=InMemoryDocstore(),
        index_to_docstore_id={},
        normalize_L2=False
    )

def aggregate_vstores(vectorstores):
    ## Initialize an empty FAISS Index and merge others into it
    ## We'll use default_faiss for simplicity, though it's tied to your embedder by reference
    agg_vstore = default_FAISS()
    for vstore in vectorstores:
        agg_vstore.merge_from(vstore)
    return agg_vstore

## Unintuitive optimization; merge_from seems to optimize constituent vector stores away
docstore = aggregate_vstores(vecstores)

print(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")

Constructed aggregate docstore with 613 chunks


<br>

### **Task 3:** Implement Your RAG Chain

Finally, all the puzzle pieces are in place to implement the RAG pipeline! As a review, we now have:

- A way to construct a from-scratch vector store for conversational memory (and a way to initialize an empty one with `default_FAISS()`)

- A vector store pre-loaded with useful document information from our `ArxivLoader` utility (stored in `docstore`).

With the help of a couple more utilities, you're finally ready to integrate your chain! A few additional convenience utilities are provided (`doc2str` and the now-common `RPrint`) but are optional to use. Additionally, some starter prompts and structures are also defined.

> **Given all of this:** Please implement the `retrieval_chain`.

In [29]:
from langchain.document_transformers import LongContextReorder
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables.passthrough import RunnableAssign
from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

import gradio as gr
from functools import partial
from operator import itemgetter

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1")


convstore = default_FAISS()

def save_memory_and_get_output(d, vstore):
    """Accepts 'input'/'output' dictionary and saves to convstore"""
    vstore.add_texts([
        f"User previously responded with {d.get('input')}",
        f"Agent previously responded with {d.get('output')}"
    ])
    return d.get('output')

initial_msg = (
    "Hello! I am a document chat agent here to help the user!"
    f" I have access to the following documents: {doc_string}\n\nHow can I help you?"
)

chat_prompt = ChatPromptTemplate.from_messages([("system",
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked: {input}\n\n"
    " From this, we have retrieved the following potentially-useful info: "
    " Conversation History Retrieval:\n{history}\n\n"
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational.)"
), ('user', '{input}')])

stream_chain = chat_prompt| RPrint() | instruct_llm | StrOutputParser()

retrieval_chain = (
    {'input' : (lambda x: x)}
    ## Make sure to retrieve history & context from convstore & docstore, respectively.
    | RunnableAssign({'history' : itemgetter('input') | convstore.as_retriever() | long_reorder | docs2str})
    | RunnableAssign({'context' : itemgetter('input') | docstore.as_retriever()  | long_reorder | docs2str})
    | RPrint()
)

def chat_gen(message, history=[], return_buffer=True):
    buffer = ""
    ## First perform the retrieval based on the input message
    retrieval = retrieval_chain.invoke(message)
    line_buffer = ""

    ## Then, stream the results of the stream_chain
    for token in stream_chain.stream(retrieval):
        buffer += token
        ## If you're using standard print, keep line from getting too long
        yield buffer if return_buffer else token

    ## Lastly, save the chat exchange to the conversation memory buffer
    save_memory_and_get_output({'input':  message, 'output': buffer}, convstore)


## Start of Agent Event Loop
test_question = "Tell me a joke."  ## <- modify as desired

## Before you launch your gradio interface, make sure your thing works
for response in chat_gen(test_question, return_buffer=False):
    print(response, end='')

{
    'input': 'Tell me a joke.',
    'history': '',
    'context': '[Quote from ReAct: Synergizing Reasoning and Acting in Language Models] > clean knife 1 with 
sinkbasin 1\nNothing happens.\n> go to countertop 3\nOn the countertop 3, you see a bread 3, a butterknife 2, a 
cellphone 1, a creditcard\n1, a fork 2, a houseplant 1, a knife 2, a spatula 1, a statue 3, a tomato 3, a 
tomato\n2, a tomato 1, and a vase 2.\n> take knife 2 from countertop 3\nNothing happens.\n> go to countertop 2\nOn 
the countertop 2, you see a cup 1, a dishsponge 1, a glassbottle 3, a plate 2, a\npotato 3, and a statue 1.\n> take
knife 1 from countertop 2\nNothing happens.\n> go to countertop 1\nOn the countertop 1, you see a lettuce 2, a mug 
2, a peppershaker 1, and a spoon 2.\n> clean knife 1 with sinkbasin 1\nNothing happens.\n> go to countertop 3\nOn 
the countertop 3, you see a bread 3, a butterknife 2, a cellphone 1, a creditcard\n1, a fork 2, a houseplant 1, a 
knife 2, a spatula 1, a statue 3, a tomato 3, a tomato\n2, a tomato 1, and a vase 2.\n> take knife 2 from 
countertop 3\nNothing happens.\n> go to countertop 2\n[Quote from ReAct: Synergizing Reasoning and Acting in 
Language Models] 1.\nYour task is to:\nput a clean knife in countertop.\n> think:\nTo solve the task, I need to 
find and take a knife, then clean it with\nsinkbasin, then put it in countertop.\nOK.\n> think:\nFirst I need to 
find a knife.\nA knife is more likely to appear in cabinet\n(1-6), drawer (1-3), countertop (1-3), fridge (1), 
garbagecan (1), shelf (1-3),\nsinkbasin (1), stoveburner (1-4), toaster (1).\nI can check one by one, starting 
with\ncabinet 1.\nOK.\n> go to cabinet 1\nOn the cabinet 1, you see a bowl 1.\n> go to cabinet 2\nThe cabinet 2 is 
closed.\n> go to cabinet 3\nOn the cabinet 3, you see a glassbottle 1.\n> go to cabinet 4\nOn the cabinet 4, you 
see a mug 1.\n> go to cabinet 5\nThe cabinet 5 is closed.\n> go to cabinet 6\nThe cabinet 6 is closed.\n> go to 
drawer 1\nThe drawer 1 is closed.\n> go to drawer 2\nThe drawer 2 is closed.\n> go to drawer 3\nThe drawer 3 is 
closed.\n> go to countertop 1\nOn the countertop 1, you see a lettuce 2, a mug 2, a peppershaker 1, and a spoon 
2.\n> go to countertop 2\n[Quote from ReAct: Synergizing Reasoning and Acting in Language Models] You are in the 
middle of a room.\nLooking quickly around you, you see a cabinet 6, a\ncabinet 5, a cabinet 4, a cabinet 3, a 
cabinet 2, a cabinet 1, a coffeemachine 1, a\ncountertop 3, a countertop 2, a countertop 1, a drawer 3, a drawer 2,
a drawer 1, a\nfridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin\n1, a 
stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster\n1.\nYour task is to:\nput a clean 
knife in countertop.\n> think:\nTo solve the task, I need to find and take a clean knife, then put it 
in\ncountertop.\nOK.\n> think:\nI need to find a clean knife.\nOK.\n> go to countertop 1\nOn the countertop 1, you 
see a lettuce 2, a mug 2, a peppershaker 1, and a spoon 2.\n> think:\nI need to find a clean knife.\nOK.\n> go to 
countertop 2\nOn the countertop 2, you see a cup 1, a dishsponge 1, a glassbottle 3, a knife 1, a\nplate 2, a 
potato 3, and a statue 1.\n> think:\nI need to find a clean knife.\nOK.\n> take knife 1 from countertop 2\n[Quote 
from ReAct: Synergizing Reasoning and Acting in Language Models] > go to countertop 2\nOn the countertop 2, you see
a cup 1, a dishsponge 1, a glassbottle 3, a knife 1, a\nplate 2, a potato 3, and a statue 1.\n> take knife 1 from 
countertop 2\nYou pick up the knife 1 from the countertop 2.\n> go to countertop 1\nOn the countertop 1, you see a 
lettuce 2, a mug 2, a peppershaker 1, and a spoon 2.\n> clean knife 1 with sinkbasin 1\nNothing happens.\n> go to 
countertop 3\nOn the countertop 3, you see a bread 3, a butterknife 2, a cellphone 1, a creditcard\n1, a fork 2, a 
houseplant 1, a knife 2, a spatula 1, a statue 3, a tomato 3, a tomato\n2, a tomato 1, and a vase 2

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: Tell me a joke.\n\n From this, we have retrieved the following potentially-useful info:  Conversation 
History Retrieval:\n\n\n Document Retrieval:\n[Quote from ReAct: Synergizing Reasoning and Acting in Language 
Models] > clean knife 1 with sinkbasin 1\nNothing happens.\n> go to countertop 3\nOn the countertop 3, you see a 
bread 3, a butterknife 2, a cellphone 1, a creditcard\n1, a fork 2, a houseplant 1, a knife 2, a spatula 1, a 
statue 3, a tomato 3, a tomato\n2, a tomato 1, and a vase 2.\n> take knife 2 from countertop 3\nNothing happens.\n>
go to countertop 2\nOn the countertop 2, you see a cup 1, a dishsponge 1, a glassbottle 3, a plate 2, a\npotato 3, 
and a statue 1.\n> take knife 1 from countertop 2\nNothing happens.\n> go to countertop 1\nOn the countertop 1, you
see a lettuce 2, a mug 2, a peppershaker 1, and a spoon 2.\n> clean knife 1 with sinkbasin 1\nNothing happens.\n> 
go to countertop 3\nOn the countertop 3, you see a bread 3, a butterknife 2, a cellphone 1, a creditcard\n1, a fork
2, a houseplant 1, a knife 2, a spatula 1, a statue 3, a tomato 3, a tomato\n2, a tomato 1, and a vase 2.\n> take 
knife 2 from countertop 3\nNothing happens.\n> go to countertop 2\n[Quote from ReAct: Synergizing Reasoning and 
Acting in Language Models] 1.\nYour task is to:\nput a clean knife in countertop.\n> think:\nTo solve the task, I 
need to find and take a knife, then clean it with\nsinkbasin, then put it in countertop.\nOK.\n> think:\nFirst I 
need to find a knife.\nA knife is more likely to appear in cabinet\n(1-6), drawer (1-3), countertop (1-3), fridge 
(1), garbagecan (1), shelf (1-3),\nsinkbasin (1), stoveburner (1-4), toaster (1).\nI can check one by one, starting
with\ncabinet 1.\nOK.\n> go to cabinet 1\nOn the cabinet 1, you see a bowl 1.\n> go to cabinet 2\nThe cabinet 2 is 
closed.\n> go to cabinet 3\nOn the cabinet 3, you see a glassbottle 1.\n> go to cabinet 4\nOn the cabinet 4, you 
see a mug 1.\n> go to cabinet 5\nThe cabinet 5 is closed.\n> go to cabinet 6\nThe cabinet 6 is closed.\n> go to 
drawer 1\nThe drawer 1 is closed.\n> go to drawer 2\nThe drawer 2 is closed.\n> go to drawer 3\nThe drawer 3 is 
closed.\n> go to countertop 1\nOn the countertop 1, you see a lettuce 2, a mug 2, a peppershaker 1, and a spoon 
2.\n> go to countertop 2\n[Quote from ReAct: Synergizing Reasoning and Acting in Language Models] You are in the 
middle of a room.\nLooking quickly around you, you see a cabinet 6, a\ncabinet 5, a cabinet 4, a cabinet 3, a 
cabinet 2, a cabinet 1, a coffeemachine 1, a\ncountertop 3, a countertop 2, a countertop 1, a drawer 3, a drawer 2,
a drawer 1, a\nfridge 1, a garbagecan 1, a microwave 1, a shelf 3, a shelf 2, a shelf 1, a sinkbasin\n1, a 
stoveburner 4, a stoveburner 3, a stoveburner 2, a stoveburner 1, and a toaster\n1.\nYour task is to:\nput a clean 
knife in countertop.\n> think:\nTo solve the task, I need to find and take a clean knife, then put it 
in\ncountertop.\nOK.\n> think:\nI need to find a clean knife.\nOK.\n> go to countertop 1\nOn the countertop 1, you 
see a lettuce 2, a mug 2, a peppershaker 1, and a spoon 2.\n> think:\nI need to find a clean knife.\nOK.\n> go to 
countertop 2\nOn the countertop 2, you see a cup 1, a dishsponge 1, a glassbottle 3, a knife 1, a\nplate 2, a 
potato 3, and a statue 1.\n> think:\nI need to find a clean knife.\nOK.\n> take knife 1 from countertop 2\n[Quote 
from ReAct: Synergizing Reasoning and Acting in Language Models] > go to countertop 2\nOn the countertop 2, you see
a cup 1, a dishsponge 1, a glassbottle 3, a knife 1, a\nplate 2, a potato 3, and a statue 1.\n> take knife 1 from 
countertop 2\nYou pick up the knife 1 from the countertop 2.\n> go to countertop 1\nOn the countertop 1, you see a 
lettuce 2, a mug 2, a peppershaker 1, and a spoon 2.\n> clean knife 1 

I'm here to help answer questions about documents, but I can certainly share a joke with you! Here it is:

Why don't scientists trust atoms?

Because they make up everything!

(I didn't find any relevant information in the provided document for this joke, it's just a classic one I thought I'd share.)

### **Task 4:** Interact With Your Gradio Chatbot

In [30]:
chatbot = gr.Chatbot(value = [[None, initial_msg]])
demo = gr.ChatInterface(chat_gen, chatbot=chatbot).queue()

try:
    demo.launch(debug=True, share=True, show_api=False)
    demo.close()
except Exception as e:
    demo.close()
    print(e)
    raise e

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3d245c691af92680ca.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


{
    'input': 'who is the main author of Mistral 7B paper?',
    'history': "[Quote from Document] User previously responded with Tell me a joke.\n[Quote from Document] Agent 
previously responded with I'm here to help answer questions about documents, but I can certainly share a joke with 
you! Here it is:\n\nWhy don't scientists trust atoms?\n\nBecause they make up everything!\n\n(I didn't find any 
relevant information in the provided document for this joke, it's just a classic one I thought I'd share.)\n",
    'context': '[Quote from Mistral 7B] . Mistral 7B outperforms the previous best 13B model (Llama 2, [26]) across
all tested\\nbenchmarks, and surpasses the best 34B model (LLaMa 34B, [25]) in mathematics and code\\ngeneration. 
Furthermore, Mistral 7B approaches the coding performance of Code-Llama 7B [20],\\nwithout sacrificing performance 
on non-code related benchmarks.\\nMistral 7B leverages grouped-query attention (GQA) [1], and sliding window 
attention (SWA) [6, 3].\\nGQA significantly accelerates the inference speed, and also reduces the memory 
requirement during\\ndecoding, allowing for higher batch sizes hence higher throughput, a crucial factor for 
real-time\\napplications. In addition, SWA is designed to handle longer sequences more effectively at a 
reduced\\ncomputational cost, thereby alleviating a common limitation in LLMs. These attention 
mechanisms\\ncollectively contribute to the enhanced performance and efficiency of Mistral 7B.\\narXiv:2310.06825v1
[cs.CL]  10 Oct 2023\\nMistral 7B is released under the Apache 2\n[Quote from Document] {\'Published\': 
\'2023-10-10\', \'Title\': \'Mistral 7B\', \'Authors\': \'Albert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, 
Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, Guillaume Lample, 
Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut Lavril, Thomas Wang,
Timothée Lacroix, William El Sayed\', \'Summary\': \'We introduce Mistral 7B v0.1, a 7-billion-parameter language 
model engineered\\nfor superior performance and efficiency. Mistral 7B outperforms Llama 2 13B\\nacross all 
evaluated benchmarks, and Llama 1 34B in reasoning, mathematics, and\\ncode generation. Our model leverages 
grouped-query attention (GQA) for faster\\ninference, coupled with sliding window attention (SWA) to effectively 
handle\\nsequences of arbitrary length with a reduced inference cost. We also provide a\\nmodel fine-tuned to 
follow instructions, Mistral 7B -- Instruct, that surpasses\\nthe Llama 2 13B -- Chat model both on human and 
automated benchmarks. Our\\nmodels are released under the Apache 2.0 license.\'}\n[Quote from Mistral 7B] .\\nNo 
proprietary data or training tricks were utilized:\\nMistral 7B \\u2013 Instruct model is a simple 
and\\npreliminary demonstration that the base model can\\neasily be fine-tuned to achieve good performance.\\nIn 
Table 3, we observe that the resulting model,\\nMistral 7B \\u2013 Instruct, exhibits superior perfor-\\nmance 
compared to all 7B models on MT-Bench,\\nand is comparable to 13B \\u2013 Chat models. An\\nindependent human 
evaluation was conducted on\\nhttps://llmboxing.com/leaderboard.\\nIn this evaluation, participants were provided 
with a set of questions along with anonymous responses\\nfrom two models and were asked to select their preferred 
response, as illustrated in Figure 6. As of\\nOctober 6, 2023, the outputs generated by Mistral 7B were preferred 
5020 times, compared to 4143\\ntimes for Llama 2 13B.\\n4\\nFigure 5: Results on MMLU, commonsense reasoning, world
knowledge and reading comprehension for\\nMistral 7B and Llama 2 (7B/13B/70B)\n[Quote from Mistral 7B] "Mistral 
7B\\nAlbert Q. Jiang, Alexandre Sablayrolles, Arthur Mensch, Chris Bamford,\\nDevendra Singh Chaplot, Diego de las 
Casas, Florian Bressand, Gianna Lengyel,\\nGuillaume Lample, Lucile Saulnier, L\\u00e9lio Renard Lavaud, Marie-Anne
Lachaux,\\nPierre S

ChatPromptValue(
    messages=[
        SystemMessage(
            content='You are a document chatbot. Help the user as they ask questions about documents. User messaged
just asked: who is the main author of Mistral 7B paper?\n\n From this, we have retrieved the following 
potentially-useful info:  Conversation History Retrieval:\n[Quote from Document] User previously responded with 
Tell me a joke.\n[Quote from Document] Agent previously responded with I\'m here to help answer questions about 
documents, but I can certainly share a joke with you! Here it is:\n\nWhy don\'t scientists trust atoms?\n\nBecause 
they make up everything!\n\n(I didn\'t find any relevant information in the provided document for this joke, it\'s 
just a classic one I thought I\'d share.)\n\n\n Document Retrieval:\n[Quote from Mistral 7B] . Mistral 7B 
outperforms the previous best 13B model (Llama 2, [26]) across all tested\\nbenchmarks, and surpasses the best 34B 
model (LLaMa 34B, [25]) in mathematics and code\\ngeneration. Furthermore, Mistral 7B approaches the coding 
performance of Code-Llama 7B [20],\\nwithout sacrificing performance on non-code related benchmarks.\\nMistral 7B 
leverages grouped-query attention (GQA) [1], and sliding window attention (SWA) [6, 3].\\nGQA significantly 
accelerates the inference speed, and also reduces the memory requirement during\\ndecoding, allowing for higher 
batch sizes hence higher throughput, a crucial factor for real-time\\napplications. In addition, SWA is designed to
handle longer sequences more effectively at a reduced\\ncomputational cost, thereby alleviating a common limitation
in LLMs. These attention mechanisms\\ncollectively contribute to the enhanced performance and efficiency of Mistral
7B.\\narXiv:2310.06825v1  [cs.CL]  10 Oct 2023\\nMistral 7B is released under the Apache 2\n[Quote from Document] 
{\'Published\': \'2023-10-10\', \'Title\': \'Mistral 7B\', \'Authors\': \'Albert Q. Jiang, Alexandre Sablayrolles, 
Arthur Mensch, Chris Bamford, Devendra Singh Chaplot, Diego de las Casas, Florian Bressand, Gianna Lengyel, 
Guillaume Lample, Lucile Saulnier, Lélio Renard Lavaud, Marie-Anne Lachaux, Pierre Stock, Teven Le Scao, Thibaut 
Lavril, Thomas Wang, Timothée Lacroix, William El Sayed\', \'Summary\': \'We introduce Mistral 7B v0.1, a 
7-billion-parameter language model engineered\\nfor superior performance and efficiency. Mistral 7B outperforms 
Llama 2 13B\\nacross all evaluated benchmarks, and Llama 1 34B in reasoning, mathematics, and\\ncode generation. 
Our model leverages grouped-query attention (GQA) for faster\\ninference, coupled with sliding window attention 
(SWA) to effectively handle\\nsequences of arbitrary length with a reduced inference cost. We also provide 
a\\nmodel fine-tuned to follow instructions, Mistral 7B -- Instruct, that surpasses\\nthe Llama 2 13B -- Chat model
both on human and automated benchmarks. Our\\nmodels are released under the Apache 2.0 license.\'}\n[Quote from 
Mistral 7B] .\\nNo proprietary data or training tricks were utilized:\\nMistral 7B \\u2013 Instruct model is a 
simple and\\npreliminary demonstration that the base model can\\neasily be fine-tuned to achieve good 
performance.\\nIn Table 3, we observe that the resulting model,\\nMistral 7B \\u2013 Instruct, exhibits superior 
perfor-\\nmance compared to all 7B models on MT-Bench,\\nand is comparable to 13B \\u2013 Chat models. 
An\\nindependent human evaluation was conducted on\\nhttps://llmboxing.com/leaderboard.\\nIn this evaluation, 
participants were provided with a set of questions along with anonymous responses\\nfrom two models and were asked 
to select their preferred response, as illustrated in Figure 6. As of\\nOctober 6, 2023, the outputs generated by 
Mistral 7B were preferred 5020 times, compared to 4143\\ntimes for Llama 2 13B.\\n4\\nFigure 5: Results on MMLU, 
commonsense reasoning, world knowledge and reading comprehension for\\nMistral 7B and Llama 2 (7B/13B/70B)\n[Quote 
from Mi

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3d245c691af92680ca.gradio.live
Closing server running on port: 7860


<br>

----

<br>

## **Part 4:** Saving Your Index For Evaluation

After you've implemented your RAG chain, please save your accumulated vector store as shown [in the official documentation](https://python.langchain.com/docs/integrations/vectorstores/faiss#saving-and-loading). You'll have a chance to use it again for your final assessment!

In [31]:
## Save and compress your index
docstore.save_local("docstore_index")
!tar czvf docstore_index.tgz docstore_index

!rm -rf docstore_index

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


If everything was properly saved, the following line can be invoked to pull the index from the compressed `tgz` file (assuming the pip requirements are installed). After you have confirmed that the cell can pull in your index, download `docstore_index.tgz` for use in the last notebook!

In [32]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

!tar xzvf docstore_index.tgz
new_db = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = new_db.similarity_search("Testing the index")
print(docs[0].page_content[:1000])

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss
. To demonstrate, we build an index using the DrQA [5]\nWikipedia dump from December 2016 and compare outputs from RAG using this index to the newer\nindex from our main results (December 2018). We prepare a list of 82 world leaders who had changed\n7\nTable 4: Human assessments for the Jeopardy\nQuestion Generation Task.\nFactuality\nSpeci\ufb01city\nBART better\n7.1%\n16.8%\nRAG better\n42.7%\n37.4%\nBoth good\n11.7%\n11.8%\nBoth poor\n17.7%\n6.9%\nNo majority\n20.8%\n20.1%\nTable 5: Ratio of distinct to total tri-grams for\ngeneration tasks.\nMSMARCO\nJeopardy QGen\nGold\n89.6%\n90.0%\nBART\n70.7%\n32.4%\nRAG-Token\n77.8%\n46.8%\nRAG-Seq.\n83.5%\n53.8%\nTable 6: Ablations on the dev set. As FEVER is a classi\ufb01cation task, both RAG models are equivalent.\nModel\nNQ\nTQA\nWQ\nCT\nJeopardy-QGen\nMSMarco\nFVR-3\nFVR-2\nExact Match\nB-1\nQB-1\nR-L\nB-1\nLabel Accuracy\nRAG-Token-BM25\n29.7\n41.5\n32.1\n33.1\n17.5\n22